In [1]:
import torch
import pandas as pd
import time
import re

In [2]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
dev

device(type='cuda')

In [3]:
oneg = pd.read_csv('../onegin.txt', sep='\n', header=None)

In [4]:
# читаем стихи
max_len = 0
phrases = []
for ind in range(len(oneg)):
    line = oneg[0][ind]
    line = line.lower()
    if re.search(r'\t', line):
        # удалить ^\t\t
        line = re.sub(r'\t\t', '', line)
        # удалить …………
        line = re.sub(r'…', '', line)
        # удалить все после [d]
        line = re.sub(r'\[\d*\].*$', '', line)
        # удалить все в квадратных скобках
        line = re.sub(r'\[.*\]', '', line)
        # удалить все анлгийские буквы
        line = re.sub(r'[abcdefghijklmnopqrstuvwxyz]', '', line)
        # удалить \xa0
        line = re.sub(r'\xa0', ' ', line)
        # удалить ведущие и ведомые пробелы
        line = re.sub(r'^ *', '', line)
        line = re.sub(r' *$', '', line)
        # удалить знаки препинания
        line = re.sub(r'[\,\.\;\:\-\(\)\!\<\>\'\«\»\—/?]', '', line)
              
        if len(line) > 0:
            phrases.append(line)
            line_len = len(line)
            if line_len > max_len:
                max_len = line_len

In [5]:
max_len

34

In [6]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [7]:
CHARS = set(' абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [8]:
MAX_LEN = max_len+1

In [9]:
X = torch.zeros((len(text), MAX_LEN), dtype=int)
X = X.to(dev)

In [10]:
X.shape

torch.Size([1453, 35])

In [11]:
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [12]:
X[0:1]

tensor([[23, 33, 12,  1,  7, 17, 19,  6, 18, 25, 33, 14, 17, 12,  4,  2,  8, 17,
         15, 28,  3, 28,  4, 29,  8, 27,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
       device='cuda:0')

In [13]:
class Network(torch.nn.Module):

    def __init__(self, dev):
        super(Network, self).__init__()
        self.dev = dev
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 100).to(self.dev)
        self.gru = torch.nn.RNN(100, 128, num_layers = 2, nonlinearity = 'relu', batch_first=True).to(self.dev)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR)).to(self.dev)

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [14]:
model = Network(dev)
model.forward(X[0:1])[0].shape

torch.Size([1, 35, 35])

In [15]:
def generate_sentence():
    sentence = ['к', 'а', 'к', ' ', 'р', 'а', 'н', 'о',' ']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        X = X.to(dev)
        if i == 0:
            result, state = model.forward(X)
        else:
            result, state = model.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
    print(''.join(sentence))

In [16]:
generate_sentence()

как рано фээллэээллээллэээллээвлэээллэээлэээ


In [25]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)
#optimizer = torch.optim.Adam(model.parameters(), lr=.01)

In [33]:
count = 0

for ep in range(1000):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    
    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    count += 1
    if count == 20:
        count = 0
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        generate_sentence()

Epoch 19. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 39. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 59. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 79. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 99. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 119. Time: 0.125, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 139. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 159. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 179. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 199. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 219. Time: 0.130, Train loss: 0.142
как рано ак оано м аотта сектнре еввдаче сз 
Epoch 239. Time: 0.130, Train loss: 0.142
как ра

In [34]:
def generate_sentence2(line):
    sentence = [w for w in line]
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        X = X.to(dev)
        if i == 0:
            result, state = model.forward(X)
        else:
            result, state = model.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
    line = ''.join(sentence)
    line = re.sub(r'none.*$', '', line)
    print(line)

In [35]:
generate_sentence2('привет чувак')

привет чувакоивытоеуяслевмоеули


In [36]:
generate_sentence2(' ')

 и страсбургский пирог


In [37]:
generate_sentence2('а')

а петербург неугомонный


In [38]:
generate_sentence2('схо')

схо одинёнатьме 


In [39]:
generate_sentence2('карета')

каретаакейк


In [40]:
generate_sentence2('з')

затем что он равно зевал


In [41]:
generate_sentence2('так думал молодой')

так думал молодойак нумал молодой лиуеше огиион с


In [42]:
generate_sentence2('с новым годом ')

с новым годом  неволианлор моалажьл ию асий ос еа


In [43]:
generate_sentence2('дед ')

дед арииконталным ый


In [44]:
generate_sentence2('баба ')

баба еладпжо уросттокни
